# Subspace Creation

The next step in the detex workflow is to create a SubSpace object from a given cluster. Before we get too far into this let's look at a new data set. 

## The data set

The data we will be working with are from a waste water injection site in Ohio. An excellent article on this site was recently published. Its reference is:


Friberg, Paul A., Glenda M. Besana‐Ostman, and Ilya Dricker. "Characterization of an earthquake sequence triggered by hydraulic fracturing in Harrison County, Ohio." Seismological Research Letters (2014).


The station key, template key,  look like this:

In [ ]:
import detex
%matplotlib inline
# display station key
stakey = detex.util.readKey("StationKey.csv", "station")
stakey


In [ ]:
temkey = detex.util.readKey("TemplateKey.csv", "template")
temkey

Additionally, we know when a nearby well began injection and ceased injection

In [ ]:
import obspy
start_inject = obspy.UTCDateTime('2013-09-07T01:45:00').timestamp
stop_inject = obspy.UTCDateTime('2013-10-06T13:18:00').timestamp


## Setup
Now we need to get the data and perform the waveform clustering. This will probably take about 20 minutes depending on your internet connection.

In [ ]:
detex.getdata.makeDataDirectories()

In [ ]:
import detex
%matplotlib inline
cl = detex.createCluster()
cl.dendro()

The 10 events we have selected are very similar, so I am going to leave them in one cluster. Let's see where they plot according to the ANF catalog. 

In [ ]:
cl.plotEvents()

## Create subspace
Next we are ready to create the subspace object. The SubSpace class requires a ClusterStream instance for input. You can pass it directly to the SubSpace class via the "cl" keyword in the init method or detex will look for a pickeled instance with the default name of "clust.pkl" in the current working directly. Whenever you create a cluster object or update its correlation coeficient requirement it is pickeled (saved to disk) with the default name. In this case we will just use the default. 

In [ ]:
ss = detex.createSubSpace() # we could also use ss = detex.construct.createSubSpace()

## Attach picks
Now we can attach the picks. These are contained in a csv that is name "PhasePicks.csv" by defualt. I created the csv by calling detex.util.pickPhases. The pickPhases function uses an altered version of streamPick (https://github.com/miili/StreamPick) which is a fantistic light-weight gui for making phase picks. 


In [ ]:
ss.attachPickTimes()

## SVD and set thresholds
Next we need to call the SVD method of the SubSpace class. It will perform the singular value decomposition in order to create an ortho-normal representation of the aligned waveforms. This is done primarily as a method of dimensionality reduction. The main method for determining how many dimensions a subspace needs to represent the events that went into it well is through an average percent energy capture requirement. If you are not sure what I am talking about take a look at figure 8 of Dave Haris' [subspace paper](https://e-reports-ext.llnl.gov/pdf/335299.pdf). 

After these steps are complete the SVD function will try to determine a threshold of detection that is significant. This is done by running the subspace detector over continuous data.